In [3]:
MAIN_PATH = "/home/carlos/MasterDS/tfm"
JSON_DATA_PATH = '{}/data/json/'.format(MAIN_PATH)
CSV_DATA_PATH = '{}/data/csv/'.format(MAIN_PATH)

In [4]:
import sys
sys.path.insert(0, MAIN_PATH)

In [37]:
%load_ext autoreload
%autoreload 2
from scripts.text.article_text_processor import ArticleTextProcessor
from scripts.text.basic_text_processor import BasicTextProcessor

from scripts.conf import TEAMS

%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
from sentence_transformers import SentenceTransformer, util

# Sentence transformers

Para la tarea de elegir la frase del artículo más cercana a cada evento,
se va a probar a calcular distintas distancias sobre embeddings creados
a partir de distintos transformadores. El objetivo de este notebook es
hacer una serie de pruebas con la librería sentence_transformers, ver 
qué modelos nos pueden servir mejor y las diferentes distancias a utilizar.

La primera aproximación será utilizar directamente modelos preentrenados.

Primero, cargamos un partido de ejemplo.

In [46]:
processor = ArticleTextProcessor()
text_proc = BasicTextProcessor()

In [41]:
all_files = processor.load_json()

In [42]:
season_file = 'premier_league_2019_2020.json'
league_season_teams = TEAMS[season_file.split('.')[0]]

In [47]:
processor.league_season_teams = league_season_teams

In [43]:
match_dict = all_files[season_file]["http://www.premierleague.com/match/46975"]
events = match_dict['events']
article = match_dict['article']

## Paraphrase identification

Modelos entrenados en tareas de identificación de frases parecidas.

In [11]:
model = SentenceTransformer('distilroberta-base-paraphrase-v1')

100%|██████████| 306M/306M [00:10<00:00, 28.2MB/s] 


In [35]:
article

"Watford were relegated from the Premier League after losing 3-2 to Arsenal as Pierre-Emerick Aubameyang's brace was not enough in the race for the Golden Boot.\nAubameyang opened the scoring in the fifth minute from the penalty spot after a Video Assistant Referee review of Craig Dawson's foul on Alexandre Lacazette.\nArsenal doubled their advantage on 24 minutes through Kieran Tierney's first goal for the club.\xa0\nIt was 3-0 on 33 minutes, Aubameyang producing an overhead kick into the net.\nWatford pulled one back 10 minutes later, Troy Deeney converting a penalty after Danny Welbeck had been fouled by David Luiz.\nWelbeck fired in from Ismaila Sarr's cross to make it 3-2 in the 66th minute before Emiliano Martinez denied the former Arsenal forward an equaliser.\nArsenal rise two places to eighth on 56 points, while\xa0Watford go down in 19th with 34 points.\xa0\nSee: Arsenal report |\xa0Watford report"

Hacemos un pequeño procesado de los datos para eliminar caracteres extraños:

In [48]:
processed_sentences = processor.process_match_article(article, 'basic')

['Watford', 'were', 'relegated', 'from', 'the', 'Premier', 'League', 'after', 'losing', 'to', 'Arsenal', 'as', 'Pierre', 'Emerick', 'Aubameyang', 'brace', 'was', 'not', 'enough', 'in', 'the', 'race', 'for', 'the', 'Golden']
['Aubameyang', 'opened', 'the', 'scoring', 'in', 'the', 'fifth', 'minute', 'from', 'the', 'penalty', 'spot', 'after', 'a', 'Video', 'Assistant', 'Referee', 'review', 'of', 'Craig', 'Dawson', 'foul', 'on', 'Alexandre']
['Arsenal', 'doubled', 'their', 'advantage', 'on', 'minutes', 'through', 'Kieran', 'Tierney', 'first', 'goal', 'for', 'the', 'club']
['It', 'was', 'on', 'minutes', 'Aubameyang', 'producing', 'an', 'overhead', 'kick', 'into', 'the', 'net']
['Watford', 'pulled', 'one', 'back', 'minutes', 'Troy', 'Deeney', 'converting', 'a', 'penalty', 'after', 'Danny', 'Welbeck', 'had', 'been', 'fouled', 'by', 'David', 'Luiz']
['Welbeck', 'fired', 'in', 'from', 'Sarr', 'cross', 'to', 'make', 'it', 'in', 'the', 'minute', 'before', 'Emiliano', 'Martinez', 'denied', 'the', 

In [49]:
processed_article = [''.join(sentence) for sentence in processed_sentences]

In [50]:
processed_article

['Watford were relegated from the Premier League after losing to Arsenal as Pierre Emerick Aubameyang brace was not enough in the race for the Golden',
 'Aubameyang opened the scoring in the fifth minute from the penalty spot after a Video Assistant Referee review of Craig Dawson foul on Alexandre',
 'Arsenal doubled their advantage on minutes through Kieran Tierney first goal for the club',
 'It was on minutes Aubameyang producing an overhead kick into the net',
 'Watford pulled one back minutes Troy Deeney converting a penalty after Danny Welbeck had been fouled by David Luiz',
 'Welbeck fired in from Sarr cross to make it in the minute before Emiliano Martinez denied the former Arsenal forward an equaliser',
 'Arsenal rise two to eighth on points while Watford go down in with points',
 'See',
 'Arsenal report Watford report']

In [54]:
article_embeddings = model.encode(processed_article, convert_to_tensor=True)

Los eventos no tienen esos caracteres extraños, pero igual es buena idea 
aplicar el mismo procesamiento.

In [53]:
events_embeddings = model.encode(events, convert_to_tensor=True)

In [55]:
cosine_scores = util.pytorch_cos_sim(events_embeddings, article_embeddings)

In [57]:
type(cosine_scores)

torch.Tensor

In [58]:
cosine_scores.shape

torch.Size([105, 9])

In [59]:
len(events)

105

In [60]:
len(processed_article)

9

In [61]:
cosine_scores

tensor([[4.3020e-01, 3.6727e-01, 2.7477e-01, 2.6489e-01, 2.8603e-01, 1.4885e-01,
         1.4791e-01, 6.1314e-02, 1.7597e-01],
        [3.2830e-01, 4.6188e-01, 2.9534e-01, 3.4798e-01, 2.9195e-01, 3.6706e-01,
         2.5858e-01, 7.8988e-02, 2.6948e-01],
        [3.2617e-01, 2.9015e-01, 2.3632e-01, 2.9489e-01, 1.9146e-01, 3.4410e-01,
         1.8733e-01, 3.4334e-02, 3.0009e-01],
        [4.8093e-01, 5.4929e-01, 4.4853e-01, 5.8288e-01, 4.1867e-01, 5.3254e-01,
         3.0712e-01, 7.8688e-02, 2.9344e-01],
        [4.0785e-01, 2.8727e-01, 2.9849e-01, 3.9561e-01, 4.0484e-01, 5.0990e-01,
         1.7689e-01, 1.0518e-01, 2.1089e-01],
        [3.3910e-01, 3.3385e-01, 2.7826e-01, 3.6128e-01, 3.2664e-01, 4.6814e-01,
         1.6027e-01, 9.9206e-02, 2.0093e-01],
        [3.5115e-01, 4.4016e-01, 3.3348e-01, 4.1306e-01, 5.2228e-01, 3.1197e-01,
         1.9635e-01, 4.5672e-02, 1.9312e-01],
        [3.5051e-01, 2.1330e-01, 2.0430e-01, 2.3112e-01, 2.2001e-01, 2.8887e-01,
         1.5813e-01, 1.7767e-0

In [56]:
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

tensor([[4.3020e-01, 3.6727e-01, 2.7477e-01, 2.6489e-01, 2.8603e-01, 1.4885e-01,
         1.4791e-01, 6.1314e-02, 1.7597e-01],
        [3.2830e-01, 4.6188e-01, 2.9534e-01, 3.4798e-01, 2.9195e-01, 3.6706e-01,
         2.5858e-01, 7.8988e-02, 2.6948e-01],
        [3.2617e-01, 2.9015e-01, 2.3632e-01, 2.9489e-01, 1.9146e-01, 3.4410e-01,
         1.8733e-01, 3.4334e-02, 3.0009e-01],
        [4.8093e-01, 5.4929e-01, 4.4853e-01, 5.8288e-01, 4.1867e-01, 5.3254e-01,
         3.0712e-01, 7.8688e-02, 2.9344e-01],
        [4.0785e-01, 2.8727e-01, 2.9849e-01, 3.9561e-01, 4.0484e-01, 5.0990e-01,
         1.7689e-01, 1.0518e-01, 2.1089e-01],
        [3.3910e-01, 3.3385e-01, 2.7826e-01, 3.6128e-01, 3.2664e-01, 4.6814e-01,
         1.6027e-01, 9.9206e-02, 2.0093e-01],
        [3.5115e-01, 4.4016e-01, 3.3348e-01, 4.1306e-01, 5.2228e-01, 3.1197e-01,
         1.9635e-01, 4.5672e-02, 1.9312e-01],
        [3.5051e-01, 2.1330e-01, 2.0430e-01, 2.3112e-01, 2.2001e-01, 2.8887e-01,
         1.5813e-01, 1.7767e-0